In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')

In [2]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

In [3]:
import json

from autogen_ext.models import AzureOpenAIChatCompletionClient

# Create the client with type-checked environment variables
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True
    }
)

In [4]:
from autogen_core.components.models import UserMessage

# Define the user message
messages = [
    UserMessage(content="What is 16 + 32?", source="user"),
]

# Call the create method on the client, passing the messages and additional arguments
# The extra_create_args dictionary includes the response format as MathReasoning model we defined above
# Providing the response format and pydantic model will use the new parse method from beta SDK
response = await client.create(messages=messages, extra_create_args={"response_format": MathReasoning})

In [5]:
from typing import Optional

# Ensure the response content is a valid JSON string before loading it
response_content: Optional[str] = response.content if isinstance(response.content, str) else None
if response_content is None:
    raise ValueError("Response content is not a valid JSON string")

# Print the response content after loading it as JSON
print(json.loads(response_content))

# Validate the response content with the MathReasoning model
MathReasoning.model_validate(json.loads(response_content))

{'steps': [{'explanation': 'To solve, we need to add the two integers together.', 'output': '16 + 32'}, {'explanation': 'Add the tens place first: 1 (tens from 16) + 3 (tens from 32) = 4 tens.', 'output': '40'}, {'explanation': 'Now add the ones place: 6 (ones from 16) + 2 (ones from 32) = 8 ones.', 'output': '8'}, {'explanation': 'Combine the results from both steps: 40 (tens) + 8 (ones).', 'output': '48'}], 'final_answer': '48'}


MathReasoning(steps=[Step(explanation='To solve, we need to add the two integers together.', output='16 + 32'), Step(explanation='Add the tens place first: 1 (tens from 16) + 3 (tens from 32) = 4 tens.', output='40'), Step(explanation='Now add the ones place: 6 (ones from 16) + 2 (ones from 32) = 8 ones.', output='8'), Step(explanation='Combine the results from both steps: 40 (tens) + 8 (ones).', output='48')], final_answer='48')